In [4]:
#imports
import pandas as pd
import numpy as np
import ipaddress

In [5]:
#merging all the csv's
df = pd.concat(
    map(pd.read_csv, 
        ['dataset/Bot-IoT/UNSW_2018_IoT_Botnet_Full5pc_1.csv',
         'dataset/Bot-IoT/UNSW_2018_IoT_Botnet_Full5pc_2.csv',
         'dataset/Bot-IoT/UNSW_2018_IoT_Botnet_Full5pc_3.csv',
         'dataset/Bot-IoT/UNSW_2018_IoT_Botnet_Full5pc_4.csv']), 
    ignore_index=True)

/tmp/ipykernel_44574/3659179818.py:2: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(


In [6]:
df.head()

,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,1,1.528089e+09,e,1,tcp,1,192.168.100.147,49960,192.168.100.7,80,...,1.12704,96,75,1.133720,1.129970,770,602,1,DoS,HTTP
1,2,1.528089e+09,e,1,arp,2,192.168.100.7,-1,192.168.100.147,-1,...,15267.20000,1,2,0.005142,0.005142,2,6,1,DoS,HTTP
2,3,1.528089e+09,e,1,tcp,1,192.168.100.147,49962,192.168.100.7,80,...,1.12704,96,75,1.135100,1.129970,770,602,1,DoS,HTTP
3,4,1.528089e+09,e,1,tcp,1,192.168.100.147,49964,192.168.100.7,80,...,1.12704,96,75,1.135140,1.129970,770,602,1,DoS,HTTP
4,5,1.528089e+09,e,1,tcp,1,192.168.100.147,49966,192.168.100.7,80,...,1.12704,96,75,1.135260,1.129970,770,602,1,DoS,HTTP


In [7]:
#typecasting into proper format
df['stime'] = df['stime'].astype('int64') 
df['ltime'] = df['ltime'].astype('int64') 

In [8]:
#taking important features
df = df[['spkts','dpkts','sbytes','dbytes','dur','proto','attack','stime','ltime','sport','dport','saddr','daddr','category']]

In [9]:
df.head()

,spkts,dpkts,sbytes,dbytes,dur,proto,attack,stime,ltime,sport,dport,saddr,daddr,category
0,5,3,650,1330,7.056393,tcp,1,1528088521,1528088528,49960,80,192.168.100.147,192.168.100.7,DoS
1,1,1,60,60,0.000131,arp,1,1528088521,1528088521,-1,-1,192.168.100.7,192.168.100.147,DoS
2,5,3,796,1330,7.047852,tcp,1,1528088521,1528088528,49962,80,192.168.100.147,192.168.100.7,DoS
3,5,3,694,1330,7.047592,tcp,1,1528088521,1528088528,49964,80,192.168.100.147,192.168.100.7,DoS
4,5,3,989,1330,7.046841,tcp,1,1528088521,1528088528,49966,80,192.168.100.147,192.168.100.7,DoS


In [10]:
#converting ip address into intgers
df['saddr'] =  df['saddr'].apply(lambda x:int(ipaddress.ip_address(x)))
df['daddr'] =  df['daddr'].apply(lambda x:int(ipaddress.ip_address(x)))

In [11]:
df.head()

,spkts,dpkts,sbytes,dbytes,dur,proto,attack,stime,ltime,sport,dport,saddr,daddr,category
0,5,3,650,1330,7.056393,tcp,1,1528088521,1528088528,49960,80,3232261267,3232261127,DoS
1,1,1,60,60,0.000131,arp,1,1528088521,1528088521,-1,-1,3232261127,3232261267,DoS
2,5,3,796,1330,7.047852,tcp,1,1528088521,1528088528,49962,80,3232261267,3232261127,DoS
3,5,3,694,1330,7.047592,tcp,1,1528088521,1528088528,49964,80,3232261267,3232261127,DoS
4,5,3,989,1330,7.046841,tcp,1,1528088521,1528088528,49966,80,3232261267,3232261127,DoS


In [12]:
df['proto'].value_counts()

tcp     617260
udp     382699
arp         33
icmp         8
Name: proto, dtype: int64

In [13]:
(1996437+1662812)/3668522 

0.9974722790268125

In [14]:
#we only take tcp and udp as by removing other columns we are only rejecting 1% of the data
df = df[(df['proto']=='udp') | (df['proto']=='tcp')]

In [15]:
#replacing udp with 0 and tcp with 1
df['proto'] = np.where(df['proto'] == 'udp', 0, 1)

In [16]:
df['category'].value_counts()

DoS    999959
Name: category, dtype: int64

In [17]:
#we only need  attack category of dos and ddos type
df = df[(df['category']=='DDoS') | (df['category']=='DoS')]

In [18]:
#resetting index
df.reset_index(drop=True,inplace=True)

In [19]:
df

,spkts,dpkts,sbytes,dbytes,dur,proto,attack,stime,ltime,sport,dport,saddr,daddr,category
0,5,3,650,1330,7.056393,1,1,1528088521,1528088528,49960,80,3232261267,3232261127,DoS
1,5,3,796,1330,7.047852,1,1,1528088521,1528088528,49962,80,3232261267,3232261127,DoS
2,5,3,694,1330,7.047592,1,1,1528088521,1528088528,49964,80,3232261267,3232261127,DoS
3,5,3,989,1330,7.046841,1,1,1528088521,1528088528,49966,80,3232261267,3232261127,DoS
4,5,3,653,1330,7.046596,1,1,1528088521,1528088528,49968,80,3232261267,3232261127,DoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999954,8,0,480,0,25.001644,0,1,1528084952,1528084977,37148,80,3232261268,3232261126,DoS
999955,8,0,480,0,25.001644,0,1,1528084952,1528084977,37149,80,3232261268,3232261126,DoS
999956,8,0,480,0,25.001644,0,1,1528084952,1528084977,37150,80,3232261268,3232261126,DoS
999957,8,0,480,0,25.001644,0,1,1528084952,1528084977,37151,80,3232261268,3232261126,DoS


In [20]:
#splitting data of category Dos into train and test
df_dos = df[df['category'] == 'DoS']
df_dos_train = df_dos.iloc[:1600000]
df_dos_test = df_dos.iloc[1641197:]

In [21]:
#saving csv ino train and test
df_dos_train.to_csv('dataset/cleaned_dataset/train/bot_iot_dos.csv',index=False)
df_dos_test.to_csv('dataset/cleaned_dataset/test/bot_iot_dos.csv',index=False)

In [22]:
#splitting data of category DDos into train and test
df_ddos = df[df['category'] == 'DDoS']
df_ddos.reset_index(drop= True,inplace = True)
df_ddos_train = df_ddos.iloc[:1600000]
df_ddos_test = df_ddos.iloc[1917572:]

In [23]:
#saving csv ino train and test
df_ddos_train.to_csv('dataset/cleaned_dataset/train/bot_iot_ddos.csv',index=False)
df_ddos_test.to_csv('dataset/cleaned_dataset/test/bot_iot_ddos.csv',index=False)